In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI

In [2]:
import gradio as gr

In [5]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"openai apikey exists and starts with {openai_api_key[:8]}")
else:
    print(f"openai apikey not set")
if anthropic_api_key:
    print(f"anthropic apikey exists and starts with {anthropic_api_key[:7]}")
else:
    print(f"anthropic apikey not set")
if google_api_key:
    print(f"google apikey exists and starts with {google_api_key[:5]}")
else:
    print(f"google apikey not set")

openai apikey exists and starts with sk-proj-
anthropic apikey exists and starts with sk-ant-
google apikey exists and starts with AIzaS


In [6]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [7]:
system_message="you are helpful assistant"
def message_gpt(prompt):
    messages=[{"role":"system","content":system_message},
            {"role":"user","content":prompt}]
    response=openai.chat.completions.create(model="gpt-4.1-mini",messages=messages)
    return response.choices[0].message.content


In [8]:
# This can reveal the "training cut off", or the most recent date in the training data

message_gpt("What is today's date?")

"Today's date is April 27, 2024. How can I assist you further?"

# User Interface Time

In [9]:
def shout(text):
    print(f"shout has been called for input {text}")
    return text.upper()

In [10]:
shout("hi")

shout has been called for input hi


'HI'

In [12]:

gr.Interface(fn=shout,inputs="textbox",outputs="textbox",flagging_mode="never").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


shout has been called for input Hi Rama krishna


In [14]:
# Adding share=True means that it can be accessed publically
# A more permanent hosting is available using a platform called Spaces from HuggingFace, which we will touch on next week
# NOTE: Some Anti-virus software and Corporate Firewalls might not like you using share=True. 
# If you're at work on on a work network, I suggest skip this test.

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [ ]:
# Adding inbrowser=True opens up a new browser window automatically

gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


shout has been called for input Hi Rama krishna & Mamatha
shout has been called for input Give me a detailed explanation of transformer architecture


# Adding Authentication

In [16]:
gr.Interface(fn=shout, inputs="textbox", outputs="textbox", flagging_mode="never").launch(inbrowser=True, auth=("RK", "bananas"))

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


shout has been called for input Hi, There!


In [28]:
system_message="you are a helpful assistant that responds in markdown without code blocks"
message_input= gr.Textbox(label="your message:",info="Enter a message for GPT-4.1-mini",lines=7)
message_output= gr.Markdown(label="Response:")
view = gr.Interface(
    fn=message_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=["Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7870
* To create a public link, set `share=True` in `launch()`.


shout has been called for input Give me a detailed explanation of transformer architecture


In [ ]:
import time

def stream_gpt(prompt):
    messages=[{"role":"system","content":system_message},
    {"role":"user","content":prompt}]
    stream=openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=messages,
        stream=True)
    result=""
    for chunk in stream:
        content = chunk.choices[0].delta.content
        if content is not None:
            result += content
            yield result
            time.sleep(0.01)  # Small delay to ensure Gradio UI updates properly


In [30]:
message_input = gr.Textbox(label="Your message:", info="Enter a message for GPT-4.1-mini", lines=7)
message_output = gr.Markdown(label="Response:")

view = gr.Interface(
    fn=stream_gpt,
    title="GPT", 
    inputs=[message_input], 
    outputs=[message_output], 
    examples=[
        "Explain the Transformer architecture to a layperson",
        "Explain the Transformer architecture to an aspiring AI engineer",
        ], 
    flagging_mode="never"
    )
view.launch()

* Running on local URL:  http://127.0.0.1:7871
* To create a public link, set `share=True` in `launch()`.
